<a href="https://colab.research.google.com/github/jinglescode/python-signal-processing/blob/main/tutorials/Task-Related%20Component%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task-Related Component Analysis

Task-related component analysis (TRCA) is a classifier, originally for steady-state visual evoked potentials (SSVEPs) detection.

Taken from the [paper](http://ieeexplore.ieee.org/document/7904641/) abstract:
> Task-related component analysis (TRCA), which can enhance reproducibility of SSVEPs across multiple trials, was employed to improve the signal-to-noise ratio (SNR) of SSVEP signals by removing background electroencephalographic (EEG) activities. An ensemble method was further developed to integrate TRCA filters corresponding to multiple stimulation frequencies.

In [1]:
#@title 
!git clone https://github.com/jinglescode/python-signal-processing.git
%cd python-signal-processing
!pip install -r requirements.txt --quiet

Cloning into 'python-signal-processing'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 167 (delta 94), reused 102 (delta 45), pack-reused 0
Receiving objects: 100% (167/167), 22.07 MiB | 42.72 MiB/s, done.
Resolving deltas: 100% (94/94), done.
/content/python-signal-processing


In [2]:
from splearn.cross_decomposition.trca import TRCA # https://github.com/jinglescode/python-signal-processing/blob/main/splearn/cross_decomposition/trca.py
from splearn.data.sample_ssvep import SampleSSVEPData # https://github.com/jinglescode/python-signal-processing/blob/main/splearn/data/sample_ssvep.py

import numpy as np
from sklearn.metrics import accuracy_score

# Load data

In this tutorial, we load a 40-target steady-state visual evoked potentials (SSVEP) dataset recorded from a single subject. It contains 6 blocks, each block consists of 40 trials, where each trial is a target. The electroencephalogram (EEG) signals has 9 channels and 1250 sampling points.

Read more about this dataset: https://www.pnas.org/content/early/2015/10/14/1508080112.abstract.

In [3]:
data = SampleSSVEPData()
eeg = data.get_data()
labels = data.get_targets()
print("eeg.shape:", eeg.shape)
print("labels.shape:", labels.shape)

eeg.shape: (6, 40, 9, 1250)
labels.shape: (6, 40)


## Leave-One-Block-Out cross-validation

We use the Leave-One-Block-Out cross-validation approach to determine TRCA's classification performance.

In [4]:
trca_classifier = TRCA(sampling_rate=data.sampling_rate)
test_accuracies = trca_classifier.leave_one_block_evaluation(eeg, labels)

Block: 1 | Train acc: 100.00% | Test acc: 97.50%
Block: 2 | Train acc: 100.00% | Test acc: 100.00%
Block: 3 | Train acc: 100.00% | Test acc: 100.00%
Block: 4 | Train acc: 100.00% | Test acc: 100.00%
Block: 5 | Train acc: 100.00% | Test acc: 97.50%
Block: 6 | Train acc: 100.00% | Test acc: 100.00%
Mean test accuracy: 99.2%


## Using `.fit` and `.predict`

In this example, we select the first 2 blocks for training and the remaining 4 blocks for testing. 

In [5]:
trca_classifier = TRCA(sampling_rate=data.sampling_rate)

x_train = eeg[0:2]
y_train = labels[0:2]

blocks, targets, channels, samples = x_train.shape
x_train = x_train.reshape((blocks-1*targets, channels, samples))
y_train = y_train.reshape((blocks-1*targets))

print("Train shape:", x_train.shape, y_train.shape)
trca_classifier.fit(x_train, y_train)

for block_i in range(2, 6):

    test_x = eeg[block_i]
    test_y = labels[block_i]

    # Shuffle the test set
    arrangement = np.arange(40)
    np.random.shuffle(arrangement)
    test_x = test_x[arrangement, :,:]
    test_y = test_y[arrangement]

    # Preduct
    pred = trca_classifier.predict(test_x)
    acc = accuracy_score(test_y, pred)

    print(f'Block: {block_i+1} | accuracy: {acc*100:.2f}%')

Train shape: (80, 9, 1250) (80,)
Block: 3 | accuracy: 100.00%
Block: 4 | accuracy: 100.00%
Block: 5 | accuracy: 97.50%
Block: 6 | accuracy: 100.00%
